In [1]:
#!/usr/bin/python
#coding=utf-8

### OSM-overpass服务接口使用，在线查询[OpenStreetMap](http:www.openstreetmap.org)开放空间数据库。
**_ by [openthings@163.com](http://my.oschina.net/u/2306127/blog), 2016-04-23. _**  
>#### overpy-使用overpass api接口的python library，这里将返回结果集保存为JSON格式。  
* 安装：$ pip install overpy
* 文档：http://python-overpy.readthedocs.org/en/latest/example.html#basic-example  
* 接口：http://wiki.openstreetmap.org/wiki/Overpass_API  
本工具例程基于上述文档例程进行编写。 

In [17]:
import os, sys, gc
import time
import json

import overpy
from pprint import *

#### 调用overpass接口，获取result数据结果集。
* 由于通过网络返回，容易中断，而且是在内存中处理，不适合创建大的查询集。

In [8]:
#范围：纬度1，经度1，纬度2，经度2
#返回：result
def get_osm():
    query = "[out:json];node(50.745,7.17,50.75,7.18);out;"
    osm_op_api = overpy.Overpass()
    result = osm_op_api.query(query)

    print("Nodes: ",len(result.nodes))
    print("Ways: ",len(result.ways))
    print("Relations: ",len(result.relations))
    return result

#### 在线获取osm数据.

In [9]:
result = get_osm()

Nodes:  2267
Ways:  0
Relations:  0


#### 显示node的属性信息（仅显示前3个node的信息）。

#### 遍历node的子集，该子集由上一步产生。

#### 将查询到的数据集合转换为json格式并写入json格式的文件（_ 该格式可由Spark直接载入: SQLContext.read.json()_ ）。

In [13]:
def node2json(node):
    jsonNode="{\"id\":\"%s\", \"lat\":\"%s\", \"lon\":\"%s\"}"%(node.id,node.lat,node.lon)
    return jsonNode

def node2jsonfile(fname,nodeset):
    fnode = open(fname,"w+")
    for n in nodeset:
        jn = node2json(n) + "\n"
        fnode.write(jn)
    fnode.close()
    print("Nodes:",len(nodeset),", Write to: ",fname)

#### 执行json文件保存操作。

In [15]:
node2jsonfile("../data/overpass.osm_node.json",result.nodes)    

Nodes: 2267 , Write to:  ../data/overpass.osm_node.json


#### 保存为GeoJSON格式。